In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import np_utils
import numpy as np

In [14]:
Tweets = pd.read_csv("./twitters/Tweets.csv")
Tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [15]:
Tweets.groupby(['airline_sentiment']).size()

airline_sentiment
negative    9178
neutral     3099
positive    2363
dtype: int64

In [16]:
Tweets = Tweets[Tweets['airline_sentiment_confidence'] > 0.8]
token = Tokenizer(num_words=100)
token.fit_on_texts(Tweets['text'].values)

In [17]:
X = token.texts_to_sequences(Tweets['text'].values)
X = pad_sequences(X, padding="post", maxlen=100)

In [18]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(Tweets['airline_sentiment'])
print(y)

[1 0 0 ... 0 1 0]


In [19]:
y = np_utils.to_categorical(y)
print(y)

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)
X_test

array([[18,  6, 45, ...,  0,  0,  0],
       [ 8, 14, 85, ...,  0,  0,  0],
       [16, 69, 36, ...,  0,  0,  0],
       ...,
       [ 8, 24, 48, ...,  0,  0,  0],
       [ 8, 23,  6, ...,  0,  0,  0],
       [13, 11,  7, ...,  0,  0,  0]])

In [21]:
modelo = Sequential()
modelo.add(Embedding(input_dim= len(token.word_index), output_dim=128, input_length=X.shape[1]))
modelo.add(SpatialDropout1D(0.2))
modelo.add(LSTM(units=196, dropout=0.2, recurrent_dropout=0, activation='tanh',
                recurrent_activation='sigmoid', unroll=False, use_bias=True))
modelo.add(Dense(units=3,activation="softmax"))

In [22]:
modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(modelo.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 128)          1638656   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 100, 128)         0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 196)               254800    
                                                                 
 dense_1 (Dense)             (None, 3)                 591       
                                                                 
Total params: 1,894,047
Trainable params: 1,894,047
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
modelo.fit(X_train, y_train, epochs=10, batch_size=30,verbose=True,validation_data=(X_test, y_test))

Epoch 1/10
245/245 [==============================] - 19s 22ms/step - loss: 0.8198 - accuracy: 0.7047 - val_loss: 0.8103 - val_accuracy: 0.7075
Epoch 2/10
245/245 [==============================] - 5s 19ms/step - loss: 0.8120 - accuracy: 0.7065 - val_loss: 0.8082 - val_accuracy: 0.7075
Epoch 3/10
245/245 [==============================] - 5s 19ms/step - loss: 0.8118 - accuracy: 0.7065 - val_loss: 0.8071 - val_accuracy: 0.7075
Epoch 4/10
245/245 [==============================] - 5s 19ms/step - loss: 0.8113 - accuracy: 0.7065 - val_loss: 0.8084 - val_accuracy: 0.7075
Epoch 5/10
245/245 [==============================] - 5s 21ms/step - loss: 0.8119 - accuracy: 0.7065 - val_loss: 0.8094 - val_accuracy: 0.7075
Epoch 6/10
245/245 [==============================] - 5s 19ms/step - loss: 0.8108 - accuracy: 0.7065 - val_loss: 0.8074 - val_accuracy: 0.7075
Epoch 7/10
245/245 [==============================] - 5s 19ms/step - loss: 0.8108 - accuracy: 0.7065 - val_loss: 0.8105 - val_accuracy: 0.707

In [24]:
loss, accuracy = modelo.evaluate(X_test,y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

99/99 [==============================] - 1s 7ms/step - loss: 0.8072 - accuracy: 0.7075
Loss:  0.8071567416191101
Accuracy:  0.7074570059776306


In [25]:
prev = modelo.predict(X_test)
print(prev)

99/99 [==============================] - 1s 5ms/step
[[0.7045015  0.14937973 0.14611878]
 [0.7045005  0.14938022 0.14611924]
 [0.7045005  0.14938022 0.14611924]
 ...
 [0.70450145 0.14937975 0.14611878]
 [0.70450014 0.14938053 0.14611936]
 [0.7045006  0.14938019 0.14611919]]
